<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Enhanced_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [54]:
try:
    import mip
except ImportError:
    import sys
    !{sys.executable} -m pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

# Define league settings

In [55]:
year = 2024
num_rounds = 15
num_teams = 12
allowed_positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
pos_limit = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

## Load player projections

In [56]:
weekly_projections_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/projection_data/2024/weekly_projections_{year}.csv'
weekly_projections = pd.read_csv(weekly_projections_url)
print(weekly_projections.columns)
weekly_projections_colums = ['player', 'position', 'team', 'points', 'week', 'year']
weekly_projections = weekly_projections[weekly_projections_colums].copy()
weekly_projections = weekly_projections.rename(columns={'player': 'Player'})
weekly_projections = weekly_projections.rename(columns={'position': 'POS'})
weekly_projections = weekly_projections[weekly_projections['POS'].isin(allowed_positions)].copy()

print(len(weekly_projections))
print(weekly_projections.head(10))

Index(['Unnamed: 0', 'player', 'position', 'team', 'points', 'sd_pts',
       'dropoff', 'floor', 'ceiling', 'points_vor', 'floor_vor', 'ceiling_vor',
       'rank', 'floor_rank', 'ceiling_rank', 'position_rank', 'tier', 'adp',
       'aav', 'uncertainty', 'week', 'year'],
      dtype='object')
4982
                Player POS team  points  week  year
0  Christian McCaffrey  RB   SF    21.4     1  2024
1          Tyreek Hill  WR  MIA    21.5     1  2024
2    Amon-Ra St. Brown  WR  DET    19.8     1  2024
3           Josh Allen  QB  BUF    24.6     1  2024
4       Bijan Robinson  RB  ATL    16.9     1  2024
5          Breece Hall  RB  NYJ    16.9     1  2024
6           Nick Chubb  RB  CLE    16.3     1  2024
7       Saquon Barkley  RB  PHI    16.3     1  2024
8     Justin Jefferson  WR  MIN    18.5     1  2024
9         Jahmyr Gibbs  RB  DET    16.0     1  2024


In [57]:
f = weekly_projections.pivot_table(
    index=['Player', 'POS'],        # Use player and position as row indices
    columns='week',                 # Weeks become columns
    values='points',                # Points are the values to fill
    aggfunc='first'                 # In case of duplicates, take the first
)

# Rename columns to 'Week X'
f.columns = [f'Week_{col}' for col in f.columns]
f['TTL'] = f.sum(axis=1)
f = f.sort_values('TTL', ascending=False)
f = f.reset_index()
f = f.fillna(0)

f = f.sort_values(['Player', 'TTL'], ascending=[True, False])
f = f.drop_duplicates(subset=['Player'], keep='first').reset_index(drop=True)

# calculate dropoff inside grouped POS
f['dropoff'] = (f.sort_values(['POS','TTL'], ascending=[True, False]).groupby('POS')['TTL'].diff(-1).fillna(0.0))
dropoff_weight = {'QB': 1.0, 'RB': 1.0, 'WR': 1.0, 'TE': 0.9, 'K': 0.4, 'DST': 0.3}
f['dropoff'] = f.apply(lambda row: row['dropoff'] * dropoff_weight.get(row['POS'], 1.0), axis=1)

# calculate value-over-replacement (VOR)
vor_dict = {}
vor_weight = {'QB': 0.8, 'RB': 1.0, 'WR': 1.0, 'TE': 0.8, 'K': 0.25, 'DST': 0.25}
for pos, limit in pos_limit.items():
    replacement_index = limit * num_teams - 1  # 0-basierte Indizierung
    pos_df = f[f['POS'] == pos].sort_values('TTL', ascending=False).reset_index(drop=True)
    if len(pos_df) > replacement_index:
        replacement_ttl = pos_df.loc[replacement_index, 'TTL']
    else:
        replacement_ttl = 0

    weight = vor_weight.get(pos, 1.0)

    for idx, row in pos_df.iterrows():
        vor = (row['TTL'] - replacement_ttl) * weight
        vor_dict[row['Player']] = row['TTL'] - replacement_ttl



f['VOR'] = f['Player'].map(vor_dict)
head_size = 15
print(len(f))
f.head(head_size)

476


,Player,POS,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,...,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,Week_18,TTL,dropoff,VOR
0,49ers,DST,5.42,5.95,5.15,5.59,5.39,5.40,4.58,5.64,...,4.54,3.89,5.59,4.54,4.67,4.11,4.32,86.01,4.263256e-15,2.07
1,A.J. Brown,WR,17.20,17.80,0.00,0.00,0.00,15.40,16.10,16.30,...,16.50,18.00,16.00,14.60,15.60,15.30,0.00,227.20,6.000000e-01,23.80
2,AJ Barner,TE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,4.07,0.00,0.00,0.00,0.00,0.00,0.00,15.97,6.930000e-01,-119.85
3,Aaron Jones,RB,12.30,12.40,12.40,14.60,14.90,0.00,14.00,15.30,...,13.70,14.60,13.90,14.10,13.90,14.40,14.10,241.60,3.600000e+00,52.62
4,Aaron Rodgers,QB,13.70,15.10,14.10,15.80,14.10,13.40,15.50,15.80,...,0.00,14.00,13.20,15.90,15.50,15.00,13.40,250.50,5.600000e+00,-29.52
5,Adam Thielen,WR,9.82,8.86,9.16,0.00,0.00,0.00,0.00,7.30,...,7.87,10.50,10.30,12.60,12.80,12.50,13.90,115.61,1.090000e+00,-87.79
6,Adam Trautman,TE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,3.39,0.00,0.00,0.00,0.00,0.00,0.00,3.39,2.700000e-01,-132.43
7,Adonai Mitchell,WR,7.60,7.02,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,7.63,0.00,0.00,0.00,0.00,0.00,29.09,2.560000e+00,-174.31
8,Aidan O'Connell,QB,0.00,0.00,0.00,3.38,2.95,11.80,12.20,0.00,...,0.00,10.40,13.20,9.94,13.90,13.30,13.00,104.07,1.450000e+00,-175.95
9,Alec Ingold,RB,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.61,0.00,1.24,0.00,0.00,0.00,0.00,2.85,1.300000e-01,-186.13


In [69]:
# filter f for player
f_filtered = f[f['Player'] == "Christian McCaffrey"]
print(f_filtered)

                 Player POS  Week_1  Week_2  Week_3  Week_4  Week_5  Week_6  \
90  Christian McCaffrey  RB    21.4     0.0     0.0     0.0     0.0     0.0   

    Week_7  Week_8  ...  Week_12  Week_13  Week_14  Week_15  Week_16  Week_17  \
90     0.0     0.0  ...     19.8     19.0      0.0      0.0      0.0      0.0   

    Week_18    TTL  dropoff    VOR  
90      0.0  101.7     0.57 -87.28  

[1 rows x 23 columns]


In [59]:
duplicates = f[f.duplicated(subset=['Player'], keep=False)]

if duplicates.empty:
    print("Keine Duplikate vorhanden")
else:
    print(duplicates.sort_values('Player'))

Keine Duplikate vorhanden


## Create Player dataset

In [60]:
players = f[['Player', 'POS', 'TTL']].copy()
players = players.rename(columns={
    'Player': 'Player',
    'POS': 'POS',
    'TTL': 'TTL'
})
players = players.sort_values('TTL', ascending=False).reset_index(drop=True)
players['Rank'] = players.index + 1

print(len(players))
print(players)

476
             Player POS     TTL  Rank
0     Lamar Jackson  QB  368.70     1
1        Josh Allen  QB  359.60     2
2    Jayden Daniels  QB  339.00     3
3     Ja'Marr Chase  WR  336.60     4
4        Joe Burrow  QB  329.70     5
..              ...  ..     ...   ...
471        C.J. Ham  RB    1.78   472
472  Frank Gore Jr.  RB    1.74   473
473  Kendall Milton  RB    1.62   474
474  Reggie Gilliam  RB    1.53   475
475       Jake Funk  RB    1.45   476

[476 rows x 4 columns]


### discarded

In [61]:
# file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
# df = pd.read_csv(file_path)#, on_bad_lines='skip')
# df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

# def extract_numbers(s):
#     if pd.isna(s):
#         return None
#     numbers = re.findall(r'\d+', str(s))
#     if numbers:
#         return int(numbers[0])
#     return None

# num_of_players = len(df) #250

# projections_df = df[['Player', 'Bye', 'POS', 'AVG']].head(num_of_players).copy()
# projections_df['Bye'] = projections_df['Bye'].apply(extract_numbers)
# projections_df['Bye'] = projections_df['Bye'].fillna(0).astype(int)

# projections_df.info()
# print(projections_df.value_counts('POS'))
# print(projections_df.head(20))

### Create custom projections
Maybe delete later when real projections available

In [62]:
# # Anzahl der Wochen
# number_of_weeks = 17

# # Wochen-Spaltennamen
# weekly_columns = [f'Week_{i+1}' for i in range(number_of_weeks)]

# # Skalierungsfunktion
# def projection_base(avg, pos, max_val=22, min_val=7, k=75, c=1.5):
#     proj = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
#     if pos == 'QB':
#         proj += 6  # QB-Bonus
#     elif pos == 'K':
#         proj -= 2  # K-Penalty
#     elif pos == 'DST':
#         proj -= 4  # DST-Penalty
#     return proj

# final_projections = []
# for _, row in projections_df.iterrows():
#     base_score = projection_base(row['AVG'], row['POS'])

#     # Erstellen der weekly projections
#     weekly_proj = []
#     for week in range(number_of_weeks):
#         # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
#         if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
#             weekly_proj.append(0.0)  # Projektion auf 0 setzen
#         else:
#             weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

#     final_projections.append(weekly_proj)

# # Projektionen in DataFrame einfügen
# f = projections_df[['Player', 'Bye', 'POS', 'AVG']].copy()
# for i, col in enumerate(weekly_columns):
#     f[col] = [proj[i] for proj in final_projections]
# f['TTL'] = f[weekly_columns].sum(axis=1)
# f['dropoff'] = (f.sort_values(['POS','TTL'], ascending=[True, False]).groupby('POS')['TTL'].diff(-1).fillna(0.0))


# # Stichprobe
# probe = 30
# print(f'Länge von f: {len(f)}')
# print(f'head({probe}) of f:')
# print()
# # print(f.head(probe))
# f.head(probe)

# Initiate Model and apply Optimization

In [73]:
# ==========================
# 1. PARAMETERS
# ==========================

players_list = players['Player'].copy().tolist()
positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
weeks = list(range(1, 18))
pos = dict(zip(weekly_projections['Player'], weekly_projections['POS']))
pos_limit = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
week_cols = [col for col in f.columns if col.startswith('Week_')]

# Build f_dict: weekly projections and dropoff for each player
f_dict = {
    row['Player']: {
        **{int(week.replace('Week_', '')): row[week] for week in week_cols},
        'dropoff': row['dropoff']
    }
    for _, row in f.iterrows()
}

beta = {t: 120.0 for t in weeks}
gamma = {'QB': 2, 'RB': 3, 'WR': 3, 'TE': 2, 'K': 1, 'DST': 1}
alpha, lambda_0, lambda_1, lambda_2, lambda_3, lambda_4 = 1.0, 1, 4, 4, 0.5, 1.5
df_sorted = players.sort_values('Rank').reset_index(drop=True)
topk_pct = 0.0025
min_pos_req = pos_limit.copy()  # Minimum roster requirements per position

# ==========================
# 2. INITIALIZATION (TEAMS, DRAFT ORDER, OPPONENT PICK)
# ==========================

teams = [f'Team {i+1}' for i in range(num_teams)]
DM_team = 'Team 4'

# Snake draft order
draft_order = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

def opponent_pick(roster, available, Rk, min_pos_req, topk_pct=0.01):
    # 1) Sort remaining players by rank
    rem = sorted(available, key=lambda p: Rk[p])
    topk = max(1, int(len(rem) * topk_pct))

    # 2) Calculate deficits per position (min requirement minus current roster)
    deficits = {
        j: min_pos_req[j] - sum(1 for p in roster if pos[p] == j)
        for j in min_pos_req
    }
    needed = [j for j, d in deficits.items() if d > 0]

    # 3) If deficits exist, pick from candidates in those positions
    if needed:
        candidates = [p for p in rem if pos[p] in needed]
        pool = candidates[:topk] if len(candidates) >= topk else candidates
        if pool:
            return random.choice(pool)

    # 4) Fallback: pick randomly from top-k overall
    return random.choice(rem[:topk])

# ==========================
# 3. DRAFT INITIALIZATION
# ==========================

rosters = {tm: [] for tm in teams}   # Dict: team -> list of drafted players
available = set(players_list)             # Set of available players
draft_log = []                       # List to store draft results

# ==========================
# 4. MAIN DRAFT LOOP
# ==========================

for pick_idx, team in enumerate(draft_order, start=1):

    # ---- 4.1: Update remaining player ranking ----
    for p in available:
      ranks = df_sorted.loc[df_sorted.Player == p, 'Rank']
      if len(ranks) != 1:
          print(f'Problem bei Spieler {p}: Anzahl gefundener Ränge = {len(ranks)}')

    rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player == p, 'Rank'].item())
    Rk = {p: i+1 for i, p in enumerate(rem)}
    player_vars = set(rem) | set(rosters[team])
    picks_remaining = num_rounds - len(rosters[team])

    # ---- 4.2: DM-Team (your team) picks via MIP ----
    if team == DM_team:
        m = Model(sense=maximize, solver_name='CBC')

        # --- Decision variables ---
        # y[i]: 1 if player i is drafted by DM, 0 otherwise
        y = {i: m.add_var(var_type=BINARY, name=f'y_{i}') for i in player_vars}
        # x[i, t]: share of player i's points used in week t (continuous)
        x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}') for i in rem for t in weeks}
        # z[t]: 1 if DM wins in week t, 0 otherwise
        z = {t: m.add_var(var_type=BINARY, name=f'z_{t}') for t in weeks}

        # --- Objective function ---
        m.objective = (
            lambda_0 * xsum(f_dict[i][t] * x[i, t] for i in rem for t in weeks) # Total points
            + lambda_1 * xsum(z[t] for t in weeks[:15])                         # Early win indicator
            # + lambda_2 * xsum(z[t] for t in weeks[15:])                         # Late win indicator
            # + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in rem)         # Dropoff bonus
            # + lambda_4 * xsum(vor_dict[i] * y[i] for i in rem)                  # VOR
        )

        # --- Constraints ---
        # Fix previous picks (already drafted players must remain picked)
        for p in rosters[team]:
            m += y[p] == 1

        # Enforce that exactly the remaining picks are made
        m += xsum(y[i] for i in rem) == picks_remaining

        # Enforce minimum requirements for each position (relative to already drafted players)
        for pos_name, req in min_pos_req.items():
            already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
            need = max(0, req - already_satisfied)
            m += xsum(y[i] for i in rem if pos[i] == pos_name) >= need

        # Weekly lineup constraints and position limits
        for j in positions:
            # m += xsum(y[i] for i in rem if pos[i] == j) >= gamma[j]  # Minimum number per position
            for t in weeks:
                m += xsum(x[i, t] for i in rem if pos[i] == j) <= pos_limit[j]  # Weekly lineup limit

        # Only drafted players can be in the weekly lineup
        for i in rem:
            for t in weeks:
                m += x[i, t] <= y[i]

        # Win indicator constraints
        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in rem) / beta[t]

        # Robust draft constraint (to simulate uncertainty in opponent picks)
        n_k = pick_idx
        for future_pick in range(
            pick_idx + 1,
            pick_idx + picks_remaining * len(teams),
            len(teams)
        ):
            top_cut = int(alpha * (future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i, r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

        # --- Solve the MIP model ---
        m.optimize()
        # Check for infeasibility
        if m.num_solutions == 0:
            raise RuntimeError(f'No feasible solution at pick {pick_idx}. Check constraints and player pool.')

        print(f"\n--- DM Pick {pick_idx} ---")
        for i in rem:
            if y[i].x is not None and y[i].x >= 0.9:
                ttl = sum(f_dict[i][t] for t in weeks)
                vor = vor_dict.get(i, 0.0)
                print(f"{i:25} ({pos[i]})  TTL={ttl:6.1f}   VOR={vor:6.1f}")


        # Extract chosen player for this pick
        chosen = [i for i in rem if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f'No feasible pick at {pick_idx}')
        pick = min(chosen, key=lambda i: Rk[i])

    # ---- 4.3: Opponent pick (simple heuristic) ----
    else:
        current_round = (pick_idx - 1) // len(teams) + 1
        dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
        pick = opponent_pick(
            roster=rosters[team],
            available=available,
            Rk=Rk,
            min_pos_req=pos_limit,
            topk_pct=dynamic_topk_pct
        )

    # ---- 4.4: Update rosters and draft log ----
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        'Pick': pick_idx,
        'Team': team,
        'Player': pick,
        'Round': (pick_idx - 1) // len(teams) + 1,
        'POS': pos[pick]
    })

# ==========================
# 5. CREATE DRAFT DATAFRAME
# ==========================

df_draft = pd.DataFrame(draft_log)
print(df_draft.head(1 + num_teams * 2))



--- DM Pick 4 ---
Dalton Kincaid            (TE)  TTL= 131.0   VOR=   0.0
Rashod Bateman            (WR)  TTL= 112.5   VOR= -82.8
Xavier Legette            (WR)  TTL= 110.1   VOR= -84.0
Nick Folk                 (K)  TTL= 106.4   VOR= -27.8
Justin Fields             (QB)  TTL= 102.5   VOR=-177.5
Vikings                   (DST)  TTL=  86.9   VOR=   7.3
Lions                     (DST)  TTL=  80.4   VOR=   1.5
Bears                     (DST)  TTL=  80.1   VOR=  -0.3
Seahawks                  (DST)  TTL=  77.4   VOR=  -1.7
Eagles                    (DST)  TTL=  77.1   VOR=  -1.7
Giants                    (DST)  TTL=  77.5   VOR=  -2.5
Falcons                   (DST)  TTL=  74.0   VOR=  -4.8
Cardinals                 (DST)  TTL=  72.8   VOR=  -6.4
Michael Carter            (RB)  TTL=   0.0   VOR=-178.2
Will Shipley              (RB)  TTL=   0.0   VOR=-179.0

--- DM Pick 21 ---
Zay Flowers               (WR)  TTL= 221.3   VOR=  33.4
Malik Nabers              (WR)  TTL= 216.7   VOR=  29.1
Ha

In [74]:
merged = pd.merge(df_draft, f[['Player', 'TTL']], on='Player', how='left')

# Replace missing TTLs (e.g., für Spieler ohne Projection) mit 0
merged['TTL'] = merged['TTL'].fillna(0)

# Group by Team and sum TTL to get total projection per team
team_ttl_proj = merged.groupby('Team')['TTL'].sum().reset_index()
team_ttl_proj = team_ttl_proj.rename(columns={'TTL': 'TTL_proj'})
team_ttl_proj

,Team,TTL_proj
0,Team 1,2780.21
1,Team 10,2947.52
2,Team 11,2747.75
3,Team 12,2853.22
4,Team 2,2913.47
5,Team 3,2910.07
6,Team 4,1400.49
7,Team 5,2925.95
8,Team 6,2811.03
9,Team 7,2956.57


## discarded

In [ ]:
# # --- 1. Parameter ---
# players        = projections_df['Player'].tolist()
# positions      = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
# weeks          = list(range(1,18))
# pos            = dict(zip(projections_df['Player'], projections_df['POS']))
# pos_limit      = {'QB':1,'RB':2,'WR':2,'TE':1,'K':1,'DST':1}
# week_cols = [col for col in f.columns if col.startswith('Week_')]
# f_dict = {
#     row['Player']: {**{int(week.replace('Week_', '')): row[week] for week in week_cols}, 'dropoff': row['dropoff']}
#     for _, row in f.iterrows()
# }
# beta           = {t:140.0 for t in weeks}
# gamma          = {'QB':2,'RB':3,'WR':3,'TE':2,'K':1,'DST':1}
# alpha, lambda_0, lambda_1, lambda_2, lambda_3 = 1.0, 1, 100, 150, 25
# df_sorted = projections_df.sort_values('AVG').reset_index(drop=True)
# df_sorted['Rank'] = df_sorted.index + 1
# topk_pct = 0.005
# min_pos_req = pos_limit


# # --- 2. Teams, DM-Team und Snake-Draft ---
# num_teams      = 12
# teams          = [f'Team {i+1}' for i in range(num_teams)]
# DM_team        = 'Team 3'
# num_rounds     = 15
# draft_order    = []
# for rnd in range(num_rounds):
#     order = teams if rnd % 2 == 0 else teams[::-1]
#     draft_order += order

# def opponent_pick(roster, available, Rk, min_pos_req, topk_pct=0.01):
#     # 1) verbleibende Spieler neu sortieren
#     rem  = sorted(available, key=lambda p: Rk[p])
#     topk = max(1, int(len(rem) * topk_pct))

#     # 2) Defizite je Position (Mindestsoll minus aktueller Bestand)
#     deficits = {
#         j: min_pos_req[j] - sum(1 for p in roster if pos[p] == j)
#         for j in min_pos_req
#     }
#     needed = [j for j, d in deficits.items() if d > 0]

#     # 3) solange Defizite bestehen, aus allen rem dieser Position picken
#     if needed:
#         # Kandidaten aller benötigten Positionen
#         candidates = [p for p in rem if pos[p] in needed]
#         # begrenze auf Top-k, falls mehr Kandidaten vorhanden
#         pool = candidates[:topk] if len(candidates) >= topk else candidates
#         if pool:
#             return random.choice(pool)

#     # 4) Fallback: zufällig aus Top-k aller Positionen
#     return random.choice(rem[:topk])

# # --- 3. Initialisierung ---
# rosters        = {tm: [] for tm in teams}
# available      = set(players)
# draft_log      = []

# # --- 4. Optimierung über alle Picks ---
# for pick_idx, team in enumerate(draft_order, start=1):
#     # 4.1 Ranking der verbleibenden Spieler aktualisieren
#     rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player==p,'AVG'].item())
#     Rk  = {p: i+1 for i,p in enumerate(rem)}
#     player_vars = set(rem) | set(rosters[team])
#     picks_remaining = num_rounds - len(rosters[team])


#     # 4.2 DM-Pick via MIP
#     if team == DM_team:
#         m = Model(sense=maximize, solver_name='CBC')

#         # Entscheidungsvariablen
#         y = {i: m.add_var(var_type=BINARY, name=f'y_{i}')
#               for i in player_vars}
#         x = {(i,t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}')
#              for i in rem for t in weeks}
#         z = {t: m.add_var(var_type=BINARY, name=f'z_{t}') for t in weeks}

#         # Objective Function
#         m.objective = (
#             lambda_0 * xsum(f_dict[i][t]*x[i,t] for i in rem for t in weeks)
#           + lambda_1 * xsum(z[t] for t in weeks[:15])
#           + lambda_2 * xsum(z[t] for t in weeks[15:])
#           + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in rem)
#         )

#         # Constraints
#         # Fixiere vergangene Picks
#         for p in rosters[team]:
#             m += y[p] == 1

#         # Exakte Anzahl verbleibender Picks
#         m += xsum(y[i] for i in rem) == picks_remaining

#         # Mindestanforderungen relativ zu schon gezogenen Spielern
#         for pos_name, req in min_pos_req.items():
#             already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
#             need = max(0, req - already_satisfied)
#             m += xsum(y[i] for i in rem if pos[i] == pos_name) >= need

#         # Position‐ und Roster‐Constraints
#         for j in positions:
#             m += xsum(y[i] for i in rem if pos[i]==j) >= gamma[j]
#             for t in weeks:
#                 m += xsum(x[i,t] for i in rem if pos[i]==j) <= pos_limit[j]

#         # # Roster Anforderungen
#         # for pos_name, req in min_pos_req.items():
#         #     m.add_constr(
#         #         xsum(y[i] for i in player_vars if pos.get(i) == pos_name) >= req,
#         #         name=f'min_roster_{pos_name}')
#         # # maximale Picks pro Team
#         # m += xsum(y[i] for i in rem) <= num_rounds

#         # nur gedraftete Spieler in der Week‐Lineup
#         for i in rem:
#             for t in weeks:
#                 m += x[i,t] <= y[i]

#         # Win‐Indicator
#         for t in weeks:
#             m += z[t] <= xsum(f_dict[i][t]*x[i,t] for i in rem) / beta[t]

#         # Robuste Draft‐Constraint
#         n_k = pick_idx
#         for future_pick in range(pick_idx+1, pick_idx + (num_rounds - len(rosters[team]))*len(teams), len(teams)):
#             top_cut = int(alpha*(future_pick - n_k))
#             if top_cut > 0:
#                 top_players = [i for i,r in Rk.items() if r <= top_cut]
#                 m += xsum(y[i] for i in top_players) <= ( (future_pick-n_k) // len(teams) )

#         m.optimize()
#         # if m.num_solutions == 0:
#         #     raise RuntimeError(f'No feasible solution at pick {pick_idx}. Check constraints and player pool.')

#         # chosen = [i for i in rem if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
#         # if not chosen:
#         #     raise RuntimeError(f'No feasible pick at {pick_idx}')
#         # pick = min(chosen, key=lambda i: Rk[i])

#         # gewählten Spieler extrahieren
#         chosen = [i for i in rem if y[i].x >= 0.99 and i not in rosters[team]]
#         if not chosen:
#             raise RuntimeError(f'No feasible pick at {pick_idx}')
#         pick = min(chosen, key=lambda i: Rk[i])

#     # 4.3 Gegner-Pick: zufällig aus Top-5 verbleibend
#     else:
#         pick = opponent_pick(roster=rosters[team], available=available, Rk=Rk, min_pos_req=pos_limit, topk_pct=topk_pct)

#     # 4.4 Update
#     rosters[team].append(pick)
#     available.remove(pick)
#     draft_log.append({
#         'Pick': pick_idx, 'Team': team, 'Player': pick,
#         'Round': (pick_idx-1)//len(teams)+1, 'POS': pos[pick]
#     })

# # --- 5. Ergebnis als DataFrame ---
# df_draft = pd.DataFrame(draft_log)
# print(df_draft.head(1+num_teams*2))

## Inspect results

In [ ]:
print(m.status)
#print(m.num_constrs, m.num_vars)
print(m)

In [ ]:
position_counts_df = df_draft.groupby(['Team', 'POS']).size().unstack(fill_value=0)
position_counts_df

In [ ]:
result_dfs = {}

for team in df_draft['Team'].unique():
    team_df = df_draft[df_draft['Team'] == team].copy()
    team_df = team_df.sort_values(by='Pick')

    # Optional: Spalten anpassen, wenn nur bestimmte Infos gewünscht sind
    team_df["Pick Info"] = team_df.apply(lambda row: f"Round {row['Round']} Pick {row['Pick']}", axis=1)
    result_dfs[team] = team_df[['Player', 'Pick Info', 'POS']]  # oder andere gewünschte Spalten

for team, df in result_dfs.items():
    print(f'=== {team} ===')
    print(df)
    print()

# Evaluation

## merge df_draft with f

In [ ]:
roster_projections = df_draft.merge(f, on='Player', how='left')

# Optional: Sortieren nach Team und Pick
roster_projections = roster_projections.sort_values(by=['Team', 'Pick']).reset_index(drop=True)
roster_projections

# create schedule

In [ ]:
num_reg_weeks = 14
playoff_weeks = [15, 16, 17]

def create_reg_schedule(teams):
    n = len(teams)
    schedule = []
    for week in range(num_reg_weeks):
        week_matches = []
        for i in range(n//2):
            team1 = teams[i]
            team2 = teams[n-1-i]
            week_matches.append((team1, team2))
        schedule.append(week_matches)
        # Rotate teams except the first one
        teams = [teams[0]] + [teams[-1]] + teams[1:-1]
    return schedule

reg_schedule = create_reg_schedule(teams)
reg_schedule

## Load real life player data

In [ ]:
data_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/Weekly_Data/weekly_data_{year}.csv'
weekly_data = pd.read_csv(data_url)
weekly_data

In [ ]:
# Roster limits
pos_limit = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

# Für jede Woche und jedes Team die beste Aufstellung bestimmen
def get_best_lineup(team, week, roster_projections, pos_limit):
    week_col = f'Week_{week}'
    team_roster = roster_projections[roster_projections['Team'] == team]
    lineup = []
    for pos, limit in pos_limit.items():
        candidates = team_roster[team_roster['POS_x'] == pos]
        # Sortiere nach projection für die Woche, nimm die besten N
        starters = candidates.sort_values(week_col, ascending=False).head(limit)
        lineup.append(starters)
    return pd.concat(lineup)

# Punkte aus weekly_data holen
def get_actual_points(lineup, week, weekly_data):
    merged = lineup.merge(
        weekly_data[weekly_data['week'] == week],
        left_on='Player', right_on='player_display_name', how='left'
    )
    # Fülle fehlende Werte (z.B. bei Bye Weeks) mit 0
    merged['fantasy_points_ppr'] = merged['fantasy_points_ppr'].fillna(0)
    return merged['fantasy_points_ppr'].sum()

# Für alle Wochen und alle Matchups durchlaufen
results = []
for week_idx, matchups in enumerate(reg_schedule, 1):
    for team1, team2 in matchups:
        lineup1 = get_best_lineup(team1, week_idx, roster_projections, pos_limit)
        lineup2 = get_best_lineup(team2, week_idx, roster_projections, pos_limit)
        points1 = get_actual_points(lineup1, week_idx, weekly_data)
        points2 = get_actual_points(lineup2, week_idx, weekly_data)
        winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
        loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
        results.append({
            'Woche': week_idx,
            'Team 1': team1,
            'Team 2': team2,
            'Punkte Team 1': points1,
            'Punkte Team 2': points2,
            'Sieger': winner,
            'Verlierer': loser
        })
df_results = pd.DataFrame(results)
df_results

In [ ]:
# Deine vorhandene Liste mit Teams
teams = [f'Team {i+1}' for i in range(num_teams)]

# Sieger und Verlierer zählen (Unentschieden ausschließen)
wins = df_results[df_results['Sieger'] != 'Unentschieden']['Sieger'].value_counts()
losses = df_results[df_results['Verlierer'] != 'Unentschieden']['Verlierer'].value_counts()

# Draws zählen: alle Teams, die in einem Unentschieden beteiligt waren
draws = (
    df_results[df_results['Sieger'] == 'Unentschieden'][['Team 1', 'Team 2']]
    .stack()
    .value_counts()
)

points_for = pd.concat([
    df_results[['Team 1', 'Punkte Team 1']].rename(columns={'Team 1': 'Team', 'Punkte Team 1': 'Points'}),
    df_results[['Team 2', 'Punkte Team 2']].rename(columns={'Team 2': 'Team', 'Punkte Team 2': 'Points'})
])
points_for = points_for.groupby('Team')['Points'].sum()

# Punkte, die jedes Team kassiert hat ("Points Against")
points_against = pd.concat([
    df_results[['Team 1', 'Punkte Team 2']].rename(columns={'Team 1': 'Team', 'Punkte Team 2': 'Points'}),
    df_results[['Team 2', 'Punkte Team 1']].rename(columns={'Team 2': 'Team', 'Punkte Team 1': 'Points'})
])
points_against = points_against.groupby('Team')['Points'].sum()

# Zusammenführen in ein DataFrame
record = pd.DataFrame({'Team': teams})
record['Wins'] = record['Team'].map(wins).fillna(0).astype(int)
record['Losses'] = record['Team'].map(losses).fillna(0).astype(int)
record['Draws'] = record['Team'].map(draws).fillna(0).astype(int)
record['Points For'] = record['Team'].map(points_for).fillna(0)
record['Points Against'] = record['Team'].map(points_against).fillna(0)
record = record.sort_values(by=['Wins', 'Draws', 'Points For'], ascending=[False, False, False]).reset_index(drop=True)

record

In [ ]:
# Setze die Playoff-Wochen
playoff_weeks = [15, 16, 17]

# Teams nach Rang sortieren (wie zuvor)
ranked_teams = record['Team'].tolist()

# Woche 15: Seed 3 vs 6, Seed 4 vs 5
week_15_matchups = [
    (ranked_teams[2], ranked_teams[5]),  # Match 1
    (ranked_teams[3], ranked_teams[4])   # Match 2
]

# Ergebnisse Woche 15
week15_results = []
for team1, team2 in week_15_matchups:
    lineup1 = get_best_lineup(team1, 15, roster_projections, pos_limit)
    lineup2 = get_best_lineup(team2, 15, roster_projections, pos_limit)
    points1 = get_actual_points(lineup1, 15, weekly_data)
    points2 = get_actual_points(lineup2, 15, weekly_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week15_results.append({
        'Woche': 15,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Extrahiere Gewinner
winners_15 = [r['Sieger'] for r in week15_results]

# Woche 16 Matchups:
# Match 3: Winner Match 1 vs Seed 2
# Match 4: Winner Match 2 vs Seed 1
week_16_matchups = [
    (winners_15[0], ranked_teams[1]),  # gegen Seed 2
    (winners_15[1], ranked_teams[0])   # gegen Seed 1
]

week16_results = []
for team1, team2 in week_16_matchups:
    lineup1 = get_best_lineup(team1, 16, roster_projections, pos_limit)
    lineup2 = get_best_lineup(team2, 16, roster_projections, pos_limit)
    points1 = get_actual_points(lineup1, 16, weekly_data)
    points2 = get_actual_points(lineup2, 16, weekly_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week16_results.append({
        'Woche': 16,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Extrahiere Gewinner
winners_16 = [r['Sieger'] for r in week16_results]

# Woche 17: Finale
week17_matchups = [(winners_16[0], winners_16[1])]

week17_results = []
for team1, team2 in week17_matchups:
    lineup1 = get_best_lineup(team1, 17, roster_projections, pos_limit)
    lineup2 = get_best_lineup(team2, 17, roster_projections, pos_limit)
    points1 = get_actual_points(lineup1, 17, weekly_data)
    points2 = get_actual_points(lineup2, 17, weekly_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week17_results.append({
        'Woche': 17,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Ergebnisse zusammenführen
df_playoff_results = pd.DataFrame(week15_results + week16_results + week17_results)

# Optional an bestehende Ergebnisse anhängen:
df_results = pd.concat([df_results, df_playoff_results], ignore_index=True)

# Finale anzeigen
champion = df_playoff_results[df_playoff_results['Woche'] == 17]['Sieger'].values[0]

In [ ]:
print(f"🏆 Der Champion ist: {champion}")
df_playoff_results